# MID-SEM CODE SUBMISSION
## NAME: ANUSHEEL KAULA                ROLL NO: 18110023
### PS: Please run the code blocks sequentially

## STANDARD CODE BLOCKS:

In [5]:
import numpy as np
from scipy.optimize import fsolve
from scipy.spatial.transform import Rotation

In [7]:
def wrapping(angle):
    if abs(angle%(2*np.pi) - 2*np.pi)<10^(-5):
        return 0
    return angle%(2*np.pi)

In [8]:
## CODE TAKEN FROM REPO
def atransformation(LinkParameters):
    a = LinkParameters[0]
    d = LinkParameters[2]
    alpha = LinkParameters[1]
    theta = LinkParameters[3]
    A = np.array([[[np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
                   [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
                   [0, np.sin(alpha), np.cos(alpha), d],
                   [0, 0, 0, 1]]])
    return (A)

def forward_kinematics_homogenous_matrix(DH):
    A = np.identity(4)
    n = np.shape(DH)[0]

    for i in range(n):
        Anext = atransformation(DH[i])
        A = np.matmul(A,Anext)

    return A

In [9]:
# https://stackoverflow.com/questions/45142959/calculate-rotation-matrix-to-align-two-vectors-in-3d-space
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

## Q1) 
## a,b & c) 
### Verifying Workspace points through forward & inverse kinematics
STANFORD MANIPULATOR:

In [10]:

link_lengths = [0.25, 0.25] # d1 = 0.25,a2 = 0.25 the lengths of the links as per question

############################### Inver Kine_ block
xc, yc, zc =0.25,0,0.35 #these are the points for which we have to check. These can be changed as required.


def Stan_Inv_Kin(xc,yc,zc,link_lengths):
    def stan_func(x):
        return [
                link_lengths[1]*np.cos(x[0]) + x[2]*np.sin(x[1])*np.sin(x[0]) - xc,
                link_lengths[1]*np.sin(x[0]) + x[2]*np.sin(x[1])*np.cos(x[0]) - yc,
                link_lengths[0] + x[2]*np.cos(x[1]) - zc
                ]
    root = fsolve(stan_func,[1,1,1])

    q1,q2,d = root

    return q1,q2,d

xyz = Stan_Inv_Kin(xc,yc,zc,link_lengths)
print(xyz)

#####################################
def stan_forwardkinematics(q1,q2,d,link_lengths):
    xc = link_lengths[1]*np.cos(q1) + d*np.sin(q2)*np.sin(q1)
    yc = link_lengths[1]*np.sin(q1) + d*np.sin(q2)*np.cos(q1)
    zc = link_lengths[0] + d*np.cos(q2)

    return xc,yc,zc

(6.018298436503189e-09, -1.5045746091265623e-08, 0.09999999999999998)


In [11]:
Stan_Inv_Kin(0.25, 0.0, 0.35,[0.25,0.25]) 

(6.018298436503189e-09, -1.5045746091265623e-08, 0.09999999999999998)

PUMA:

In [12]:

link_lengths = [0.25, 0.25, 0.25] # order : d1, a1, a2 replace d1 with link_lengths[0], a3 with link_lengths[1].
                                  # a3 with link_lengths[2]

def puma_inverse(xc,yc,zc,link_lengths):
    solutions = []
    theta1 = np.arctan2(yc,xc)
    D = (xc**2 + yc**2 + (zc-link_lengths[0])**2 -link_lengths[1]**2 - link_lengths[2]**2)/(2*link_lengths[1]*link_lengths[2])
    if abs(D)<=1:
        theta3 = np.arctan2(np.sqrt(1-D**2),D)
        theta2 = np.arctan2(zc-link_lengths[0],np.sqrt(xc**2 + yc**2)) - np.arctan2(link_lengths[2]*np.sin(theta3),link_lengths[1] + link_lengths[2]*np.cos(theta3))

        solutions.append([theta1,theta2,theta3])

        theta3 = np.arctan2(-np.sqrt(1-D**2),D)
        theta2 = np.arctan2(zc-link_lengths[0],np.sqrt(xc**2 + yc**2)) - np.arctan2(link_lengths[2]*np.sin(theta3),link_lengths[1] + link_lengths[2]*np.cos(theta3))

        solutions.append([theta1,theta2,theta3])
    else:
        print("Error. The given inputs are out of bounds of workspace")
    
    return solutions

def puma_forward(q1,q2,q3):
    xc = link_lengths[1]*np.cos(q2)*np.cos(q1) + link_lengths[2]*np.cos(q2+q3)*np.cos(q1)
    yc = link_lengths[1]*np.cos(q2)*np.sin(q1) + link_lengths[2]*np.cos(q2+q3)*np.sin(q1)
    zc = link_lengths[0] + link_lengths[1]*np.sin(q2) + link_lengths[2]*np.sin(q2+q3)

    return xc,yc,zc




In [13]:
temp = puma_inverse(0.5,0.0,0.25,[0.25,0.25,0.25]) # the first three arguements are xc, yc & zc respectively
puma_forward(temp[0][0],temp[0][1],temp[0][2])

(0.5, 0.0, 0.25)

SCARA:

In [14]:
link_lengths = [0.25, 0.25, 0.25] # d1 , a1, a2 these are the link lengths
xc,yc,zc = 0,0,0 # these are the points in space for which we have to perform the check 
                # chenge these values as required. default 0 value is assigned.
def inv_func(x):
    return [
            - xc + link_lengths[1]*np.cos(x[0]) + link_lengths[2]*np.cos(x[0]+x[1]),
            - yc + link_lengths[1]*np.sin(x[0]) + link_lengths[2]*np.sin(x[0]+x[1]),
            - zc + link_lengths[0] - x[2]
            ]
root = fsolve(inv_func,[1,1,1])

q1,q2,d = root
def scara_inverse(xc,yc,zc):
    if np.sqrt(xc**2+yc**2)>link_lengths[2] + link_lengths[1]:
        print("No Solution can be Found!")
        return []
    else: 
        def inv_func(x):
            return [
                    - xc + link_lengths[1]*np.cos(x[0]) + link_lengths[2]*np.cos(x[0]+x[1]),
                    - yc + link_lengths[1]*np.sin(x[0]) + link_lengths[2]*np.sin(x[0]+x[1]),
                    - zc + link_lengths[0] - x[2]
                    ]
        root = fsolve(inv_func,[1,1,1])

        q1,q2,d = root

        return [wrapping(q1),wrapping(q2),d]

def scara_forward(q1,q2,d):

    xc = link_lengths[1]*np.cos(q1) + link_lengths[2]*np.cos(q1+q2)
    yc = link_lengths[1]*np.sin(q1) + link_lengths[2]*np.sin(q1+q2)
    zc = link_lengths[0] - d

    return xc,yc,zc

In [16]:
t1,t2,t3 = scara_forward(np.pi/2,0,2)
k = scara_inverse(t1,t2,t3)

## Q1) 
## e & f) 
### Attaching wrist
PUMA:

In [3]:
links = [1,1,1] # link lengths are as d1 {height}, a1 {link 1}, a2 {link 2} 
def puma_wrist_inv(xt,yt,zt):
    xc=xt ; 
    yc=yt ; 
    zc=zt+0.05 ; #since the fixed distance of 5 cm has to be mainted, thus the term is added.
    
    if abs((xc**2 + yc**2 + (zc-links[0])**2 - links[1]**2 - links[2]**2)/(2*links[1]*links[2])) <= 1:
        def inv_func(x):
                return [
                        -xc + links[1]*np.cos(x[1])*np.cos(x[0]) + links[2]*np.cos(x[1]+x[2])*np.cos(x[0]),
                        -yc + links[1]*np.cos(x[1])*np.sin(x[0]) + links[2]*np.cos(x[1]+x[2])*np.sin(x[0]),
                        -zc + links[0] + links[1]*np.sin(x[1]) + links[2]*np.sin(x[1]+x[2]),
                        ]
        root = fsolve(inv_func,[1,1,1])
        q1,q2,q3 = root
        q1,q2,q3 = wrapping(q1),wrapping(q2),wrapping(q3)
        dh_param = [[0,np.pi/2,links[0],q1],
                    [links[1],0,0,q2],
                    [links[2],0,0,q3]]
        homo_mat = forward_kinematics_homogenous_matrix(dh_param)
        jc = []
        for mat in homo_mat:
            jc.append(mat[:3,-1])
        
        vec1 = jc[-1] - jc[-2]
        vec2 = [xc-xt,yc-yt,zc -zt]

        rot_mat = rotation_matrix_from_vectors(vec2,vec1) #since we know the direction of two vetors; one at the link & other at the surface
                                                    # we can obtain a rotation matrix from the two.
        r = Rotation.from_matrix(rot_mat)
        euler_ang = r.as_euler('zyx',degrees = False)
        
        joint_params = [q1,q2,q3,euler_ang[0],euler_ang[1],euler_ang[2]]

        return joint_params
    else:
        print("No solution!!")

def puma_forward_kin(q1,q2,q3,q4,q5,q6):
    dh_param_p = [[0,np.pi/2,links[0],q1],
                    [links[1],0,0,q2],
                    [links[2],0,0,q3],
                    [0,-np.pi/2,0,q4],
                    [0,np.pi/2,0,q5],
                    [0.05,0,0,q6]]
    homo_mat = forward_kinematics_homogenous_matrix(dh_param_p)
    

In [30]:
#FUNCTION CALLING
puma_wrist_inv(0.45,0.075,0.25)

IndexError: list index out of range

STANFORD:

In [24]:
links = [1,1,0.5] # link lengths d1 = 1, a1 = 1 & maximum travel of prismatic joint = 0.5
def stan_wrist_inv(xt,yt,zt):
    xc = xt ; 
    yc = yt ; 
    zc = zt+0.05 ; #since the fixed distance of 5 cm has to be mainted, thus the term is added.
    check = xc**2 + yc**2 + (zc**2 - links[0]**2) - (links[1]**2 + links[2]**2) <= 0 and abs(zc - links[0])<=links[2] and xc**2 + yc**2>=links[1]**2
    print(check)
    if check:
        def inv_func(x):
                return [
                        links[1]*np.cos(x[0]) + x[2]*np.sin(x[1])*np.sin(x[0]) - xc,
                        links[1]*np.sin(x[0]) + x[2]*np.sin(x[1])*np.cos(x[0]) - yc,
                        links[0] + x[2]*np.cos(x[1]) - zc
                        ]
        root = fsolve(inv_func,[1,1,1])
        q1,q2,d = root
        q1,q2 = wrapping(q1),wrapping(q2)
        dh_param_s = [[0,-np.pi/2,links[0],q1],
                        [0,np.pi/2,links[1],q2],links[0]
                        [0,0,d,0]]
        homo_mat_s = forward_kinematics_homogenous_matrix(dh_param_s)
        jc_s = []
        for mat in homo_mat_s:
            jc_s.append(mat[:3,-1])
        
        vec1 = jc_s[-1] - jc_s[-2]
        vec2 = [xc-xt,yc-yt,zc -zt]

        rot_mat = rotation_matrix_from_vectors(vec2,vec1) #since we know the direction of two vetors; one at the link & other at the surface
                                                    # we can obtain a rotation matrix from the two.
        r = Rotation.from_matrix(rot_mat)
        euler_ang = r.as_euler('zyx',degrees = False)
        
        joint_params = [q1,q2,q3,euler_ang[0],euler_ang[1],euler_ang[2]]

        return joint_params
    else:
        print("No solution!!")
def stan_forw_wr(q1,q2,d):
        xc = links[1]*np.cos(q1) + d*np.sin(q2)*np.sin(q1)
        yc = links[1]*np.sin(q1) + d*np.sin(q2)*np.cos(q1)
        zc = links[0] + d*np.cos(q2)

        return xc,yc,zc

SCARA:

In [29]:
links = [1,1,1] #links d1, a1, a2
def scara_inv(x,y,z):
    xc = x ; 
    yc = yc ; 
    zc = z + 0.05 ; #since the fixed distance of 5 cm has to be mainted, thus the term is added.
    if np.sqrt(xc**2+yc**2)>links[2] + links[1]:
            print("No Solution !!")
            return []
else:
        def inv_func(x):
                return [
                        - xc + links[1]*np.cos(x[0]) + links[2]*np.cos(x[0]+x[1]),
                        - yc + links[1]*np.sin(x[0]) + links[2]*np.sin(x[0]+x[1]),
                        - zc + links[0] - x[2]
                        ]
        root = fsolve(inv_func,[1,1,1])

            q1,q2,d = root
            q1,q2 = wrapping(q1),wrapping(q2)
            joint_para = [q1,q2,0,0,0,0]
            return joint_para

def scara_forw(q1,q2,d):

        xc = links[1]*np.cos(q1) + links[2]*np.cos(q1+q2)
        yc = links[1]*np.sin(q1) + links[2]*np.sin(q1+q2)
        zc = links[0] - d

        return xc,yc,zc


SyntaxError: invalid syntax (Temp/ipykernel_9052/1115957504.py, line 9)

In [ ]:
print(scara_inv(0.45,0.075,0.25))